In [1]:
pip install arxiv

     ---------------------------------------- 81.3/81.3 kB 1.1 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6060 sha256=9a196107aef7fbf78a04259f352a7dff043018a173cdab1b75a241e8c860a8b4
  Stored in directory: c:\users\isha\appdata\local\pip\cache\wheels\71\ec\b2\56e051ac90f775eec34b746094d38c9cde52e668fe9eb42ac4
Successfully built sgmllib3k
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 24.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper


In [9]:
api_wrapper= WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=2000)
wiki = WikipediaQueryRun(api_wrapper=api_wrapper)

In [13]:
wiki.name


'wikipedia'

In [8]:
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader= WebBaseLoader("https://docs.smith.langchain.com/")
docs = loader.load()

documents = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents, OllamaEmbeddings())
retreiver= vectordb.as_retriever()
retreiver

VectorStoreRetriever(tags=['FAISS', 'OllamaEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000001FD22E7A110>)

In [10]:
from langchain.tools.retriever import create_retriever_tool
retriever_tool = create_retriever_tool(retreiver, "langsmith_search", "Search for information about LangSmith. For any questions about LangSmith, you must use this tool") 


In [11]:
retriever_tool.name

'langsmith_search'

In [12]:
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper = ArxivAPIWrapper(top_k_results=1,doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [14]:
tools=[wiki,arxiv,retriever_tool]

In [26]:
# from langchain_community.llms import Ollama

# llm  = Ollama(model="llama2")
# llm

from langchain_community.chat_models import ChatOllama
## Load Ollama LAMA2 LLM model
llm=ChatOllama(model="llama2")
llm

ChatOllama()

In [17]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [28]:
from langchain.agents import create_openai_tools_agent
agent = create_openai_tools_agent(llm,tools,prompt)


In [29]:
#Agent executor
from langchain.agents import AgentExecutor
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, metadata={'lc_hub_owner': 'hwchase17', 'lc_hub_repo': 'openai-functions-agent', 'lc_hub_commit_hash': 'a165

In [31]:

agent_executor.invoke({"input": "Tell me about LangSmith"})



> Entering new AgentExecutor chain...
